In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import math
from tensorflow.python.framework import ops
import xgboost as xgb
from sklearn.model_selection import train_test_split
%matplotlib inline
import sys

In [11]:
sys.path.append('C:/Users/45115/Anaconda3/mywhl/xgboost-master/python-package')
sys.path

['',
 'C:\\Python27\\Lib',
 'C:\\Python27\\DLLs',
 'C:\\Python27\\Lib\\lib-tk',
 'C:\\other-folders-on-the-path',
 'C:\\Users\\45115\\OneDrive\\Fking-ML\\Kaggle',
 'C:\\Users\\45115\\Anaconda3\\python36.zip',
 'C:\\Users\\45115\\Anaconda3\\DLLs',
 'C:\\Users\\45115\\Anaconda3\\lib',
 'C:\\Users\\45115\\Anaconda3',
 'C:\\Users\\45115\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\45115\\Anaconda3\\lib\\site-packages\\xgboost-0.72-py3.6.egg',
 'C:\\Users\\45115\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\45115\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\45115\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\45115\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\45115\\.ipython',
 'C:/Users/45115/Anaconda3/mywhl/xgboost-master/python-package']

In [12]:
train = pd.read_csv("datasets/digit/train.csv")
test = pd.read_csv('datasets/digit/test.csv')

In [13]:
X_train=train
X_test=test
Y_train=train["label"]

# XGB

In [14]:
import time
start_time = time.time()

In [15]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train, Y_train, test_size=0.3, random_state=1)

## My xgboost

In [16]:
y = Xtrain.label
X = Xtrain.drop(['label'],axis=1)
val_y = Xtest.label
val_X = Xtest.drop(['label'],axis=1)

In [17]:
xgb_val = xgb.DMatrix(val_X, label = val_y)
xgb_train = xgb.DMatrix(X, label = y)
xgb_test = xgb.DMatrix(test)

In [18]:
params = {
    'booster':'gbtree',
'objective': 'multi:softmax', #多分类的问题
'num_class':10, # 类别数，与 multisoftmax 并用
'gamma':0.2,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':8, # 构建树的深度，越大越容易过拟合
'lambda':3,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.5, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.01, # 如同学习率
'seed':1000,
#'nthread':7,# cpu 线程数
#'eval_metric': 'auc'
'tree_method': 'gpu_hist'
}
plst = list(params.items())
num_rounds = 100
watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]

model = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100)

model.save_model('./model/xgb.model')
print("best best_ntree_limit", model.best_ntree_limit)
preds = model.predict(xgb_test, ntree_limit=model.best_ntree_limit)

np.savetxt('xgb_submission.csv', np.c_[range(1,len(tests)+1),preds],delimiter=',',header='ImageId,Label',comments='',fmt='%d')

cost_time = time.time()-start_time
print("xgboost success!",'\n',"cost time:",cost_time,"(s)......")
cost_time = time.time() - start_time
print("xgboost successful!", '\n', "cost time:", cost_time, "(s)")

XGBoostError: b'[16:47:54] src/learner.cc:153: XGBoost version not compiled with GPU support.'

best best_ntree_limit 98


## xgboost by others

In [ ]:
train_xy, val = train_test_split(train, test_size=0.3, random_state=1)

In [ ]:
print(val.head())

# tensorflow

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.values.reshape(-1)].T
    return Y

In [ ]:
X_train=X_train/255
X_test=X_test/255
X_train=X_train.values.reshape(42000,28,28,1)
X_test=X_test.values.reshape(28000,28,28,1)
print(X_train.shape)
print(X_test.shape)
Y_train=convert_to_one_hot(Y_train,10).T
print(Y_train.shape)

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [ ]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    
    X = tf.placeholder(tf.float32,[None,n_H0,n_W0,n_C0],name="X")
    Y = tf.placeholder(tf.float32,[None,n_y],name="Y")

    
    return X, Y

In [ ]:
def initialize_parameters():
    tf.set_random_seed(1)
    W1 = tf.get_variable("W1",[4,4,1,8],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2",[2,2,8,16],initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    

    parameters = {"W1": W1,
                  "W2": W2}
    
    return parameters

In [ ]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding="SAME")
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,ksize=[1,8,8,1],strides=[1,2,2,1],padding="SAME")
    Z2 = tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding="SAME")
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2,ksize=[1,4,4,1],strides=[1,4,4,1],padding="SAME")
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2,10,activation_fn=None)
    return Z3

In [ ]:
def compute_cost(Z3,Y):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3,labels=Y))
    return cost

In [ ]:
def model(X_train, Y_train,X_test, learning_rate = 0.005,
          num_epochs = 150, minibatch_size = 64, print_cost = True):
     
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []
    X, Y = create_placeholders(n_H0,n_W0,n_C0,n_y)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X,parameters)
    cost =compute_cost(Z3,Y)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):

            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y})
                minibatch_cost += temp_cost / num_minibatches
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        parameters = sess.run(parameters)
        correct_predictions = tf.equal(tf.argmax(Y, axis=1), tf.argmax(Z3, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, 'float'))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        print("Train Accuracy:", train_accuracy)
        Y_test = sess.run(tf.argmax(Z3, 1), feed_dict={X: X_test})

        return Y_test

In [ ]:
Ytest=model(X_train,Y_train,X_test)